In [2]:
import numpy as np 
import cv2 
from sklearn.cluster import DBSCAN

	
# Read the query image as query_img 
# and train image This query image 
# is what you need to find in train image 
# Save it in the same directory 
# with the name image.jpg 
query_img = cv2.imread('patch_image.png') 
train_img = cv2.imread('combined_large_image.png') 

# Convert it to grayscale 
query_img_bw = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY) 
train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY) 

# Initialize the ORB detector algorithm 
orb = cv2.ORB_create() 

# Now detect the keypoints and compute 
# the descriptors for the query image 
# and train image 
queryKeypoints, queryDescriptors = orb.detectAndCompute(query_img_bw,None) 
trainKeypoints, trainDescriptors = orb.detectAndCompute(train_img_bw,None) 

# Initialize the Matcher for matching 
# the keypoints and then match the 
# keypoints 
matcher = cv2.BFMatcher() 
matches = matcher.match(queryDescriptors,trainDescriptors)
good_matches = [m for m in matches if m.distance < 30]

# draw the matches to the final image 
# containing both the images the drawMatches() 
# function takes both images and keypoints 
# and outputs the matched query image with 
# its train image 
final_img = cv2.drawMatches(query_img, queryKeypoints, 
train_img, trainKeypoints, good_matches[:20],None) 

final_img = cv2.resize(final_img, (1000,650)) 




In [4]:
edges = cv2.Canny(train_img_bw, 100, 200)
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


In [5]:
counts = []
for contour in contours:
    count = 0
    for keypoint in queryKeypoints:
        # Get the coordinate of the keypoint
        x, y = keypoint.pt
        # Check if the keypoint is inside or on the contour
        result = cv2.pointPolygonTest(contour, (x, y), False)
        # If the result is not negative, the keypoint is inside or on the contour
        if result >= 0:
            count += 1
    # Append the count to the list
    counts.append(count)

In [6]:
counts = np.array(counts)

# Find the contour that has the maximum count of keypoints
max_index = np.argmax(counts)
max_contour = contours[max_index]

In [9]:
image_copy = train_img.copy()
cv2.drawContours(image_copy, [max_contour], -1, (0, 255, 0), 3)

# Draw the keypoints on the image
image_copy = cv2.drawKeypoints(image_copy, queryKeypoints, None, color=(0, 0, 255))

# Display the image with the contour and the keypoints
cv2.imshow("Image with Contour and Keypoints", image_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
